In [1]:
import os
import pandas as pd
import datetime
from matplotlib import pyplot
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import CuDNNLSTM
from keras.layers import CuDNNGRU
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers import Bidirectional
from keras.layers import Lambda
from keras.layers import Dropout
from keras.layers import SimpleRNN
from keras.utils import generic_utils
from keras.utils import Sequence
from keras import regularizers
from keras import optimizers
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
import numpy as np
import scipy 
import math
import h5py
import time
import copy
import sys
import feather
from sklearn.metrics import mean_absolute_error
from numpy.random import seed
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)
seed(123)
from tensorflow import set_random_seed
set_random_seed(456)
np.set_printoptions(threshold=sys.maxsize)

Using TensorFlow backend.


In [2]:
import keras
print(os.getcwd())
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
tf.keras.backend.set_session(tf.Session(config=config))
from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())

/home/whsu014/dev/mlrig/Experiments


In [3]:
# set up training set
view_1hot_df = feather.read_dataframe("/home/whsu014/data/view_1hot_nona_len29_float_age_impute_and_True_HG_100096inds.feather")
print(view_1hot_df.columns)
print(view_1hot_df.shape)

print(view_1hot_df['CVD_ISOSORBRIDE_DINITRATE'].unique())
view_1hot_df = view_1hot_df.drop(['CVD_ISOSORBRIDE_DINITRATE'], axis=1)
print(view_1hot_df.shape)
print("Number of inds:", len(view_1hot_df['VSIMPLE_INDEX_MASTER'].unique()))
# set length to 28
print(view_1hot_df.shape[0]/29)
view_1hot_df = copy.deepcopy(view_1hot_df[view_1hot_df['QUARTER']!=0])
print(view_1hot_df.shape[0]/28)
# Take 90000 inds for train and validation set
# remove true values
# remove QUARTER
view_1hot_df = view_1hot_df.drop(['QUARTER'], axis=1)
train_v = copy.deepcopy(view_1hot_df.iloc[:(28*90000), :-7].values)
print(train_v.shape)
print(train_v.shape[0]/28)

# set up test set
test_df = feather.read_dataframe("/home/whsu014/data/Test_set_with_TRUE_10096inds.feather")
print(test_df.columns)
print(test_df.shape)
print("Number of inds:", len(test_df['VSIMPLE_INDEX_MASTER'].unique()))
test_v = copy.deepcopy(test_df.iloc[:, :-7].values)

Index(['VSIMPLE_INDEX_MASTER', 'QUARTER', 'AGE', 'SEX', 'NZDEP', 'ETHN_1',
       'ETHN_2', 'ETHN_3', 'ETHN_4', 'ETHN_5',
       ...
       'PT_DIABETES_YR', 'PT_ATRIAL_FIBRILLATION', 'PT_IMP_FATAL_CVD',
       'TRUE_HDL', 'TRUE_LDL', 'TRUE_TRI', 'TRUE_TCL', 'TRUE_TCHDL',
       'TRUE_HBA1C', 'TRUE_EGFR'],
      dtype='object', length=191)
(2902784, 191)
[0.]
(2902784, 190)
Number of inds: 100096
100096.0
100096.0
(2520000, 182)
90000.0
Index(['VSIMPLE_INDEX_MASTER', 'AGE', 'SEX', 'NZDEP', 'ETHN_1', 'ETHN_2',
       'ETHN_3', 'ETHN_4', 'ETHN_5', 'TEST',
       ...
       'PT_DIABETES_YR', 'PT_ATRIAL_FIBRILLATION', 'PT_IMP_FATAL_CVD',
       'TRUE_HDL', 'TRUE_LDL', 'TRUE_TRI', 'TRUE_TCL', 'TRUE_TCHDL',
       'TRUE_HBA1C', 'TRUE_EGFR'],
      dtype='object', length=189)
(282688, 189)
Number of inds: 10096


In [5]:
# standardise data
# z = (X - u)/s
# u is the mean of the training samples
# s is the standard devidation of the training samples
scaler = StandardScaler()
scaled_train = scaler.fit_transform(train_v) 
scaled_test = scaler.transform(test_v)

In [6]:
print('LL_PDC:', view_1hot_df.columns.get_loc('LL_PDC'))

LL_PDC: 28


In [9]:
def setup_xy(time_series_v, scaled_time_series_v):
    num_of_inds = time_series_v.shape[0]//28
    n_features = time_series_v.shape[1]
    
    _x = np.empty((num_of_inds, 
                   n_features+(n_features-1)*7), 
                   dtype='float')
    _y = np.empty((num_of_inds, 2), dtype='float') # 3, 
    ###############################################
    # construct x and y  flatten times series 
    # to 1 row.
    ###############################################
    for i in range(num_of_inds):
        scaled_ind_v = copy.deepcopy(scaled_time_series_v[(i*28):((i+1)*28), :])
        _x[i, :n_features] = copy.deepcopy(scaled_ind_v[0, :])
        for j in range(1, 8):
            start_idx = (j-1)*(n_features-1)+n_features
            end_idx = start_idx + (n_features-1)
            _x[i, start_idx:end_idx] = copy.deepcopy(scaled_ind_v[j, 1:])
        ind_v = copy.deepcopy(time_series_v[(i*28):((i+1)*28), :])
        if sum(ind_v[8:28, 28])/20 >= 80:
            _y[i, 0] = 0.0
            _y[i, 1] = 1.0
        else:
            _y[i, 0] = 1.0
            _y[i, 1] = 0.0
    return _x, _y

In [10]:
x_train, y_train = setup_xy(train_v, scaled_train)
x_test, y_test = setup_xy(test_v, scaled_test)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(90000, 1449)
(90000, 2)
(10096, 1449)
(10096, 2)


In [11]:
num_per_fold = x_train.shape[0]//10
print(num_per_fold)

9000


In [12]:
    def set_up_model(n_units, dropout_rate_1, dropout_rate_2):
        model = Sequential()
        model.add(Dense(n_units, activation='relu', 
                        #kernel_regularizer=regularizers.l2(0.01),
                        input_shape=(1449, ))) #1449, 1457
        model.add(Dropout(dropout_rate_1))
        model.add(Dense(n_units, activation='relu'))
                        #kernel_regularizer=regularizers.l2(0.01)))
        model.add(Dropout(dropout_rate_2))
        model.add(Dense(2, activation='softmax')) #20
                       #kernel_regularizer=regularizers.l2(0.01)))
        #sgdm = optimizers.SGD(lr=0.001, momentum=0.9, decay=1e-4)  #1e-4
        model.compile(loss='categorical_crossentropy', 
                      optimizer='adam', metrics=['accuracy'])
        return model

In [16]:
def fit_model(model, X_train, Y_train, b_size):
    val_weight = "MLP_val_weights.hdf5"
    val_checkpointer = ModelCheckpoint(filepath=val_weight,
                                         monitor='val_loss', verbose=1,
                                         save_best_only=True)
    history = model.fit(X_train, Y_train,
                        batch_size=b_size, 
                        epochs=50, 
                        validation_split=0.1, 
                        callbacks=[val_checkpointer], 
                        verbose=2, 
                        shuffle=False)
    pyplot.plot(history.history['loss'], label='train')
    pyplot.plot(history.history['val_loss'], label='validation')
    pyplot.legend()
    pyplot.show()
    return min(history.history['val_loss']), val_weight

9000
Train on 81000 samples, validate on 9000 samples
Epoch 1/50
 - 2s - loss: 0.6000 - acc: 0.7053 - val_loss: 0.5468 - val_acc: 0.7356

Epoch 00001: val_loss improved from inf to 0.54677, saving model to MLP_val_weights.hdf5
Epoch 2/50
 - 1s - loss: 0.5460 - acc: 0.7371 - val_loss: 0.5362 - val_acc: 0.7400

Epoch 00002: val_loss improved from 0.54677 to 0.53617, saving model to MLP_val_weights.hdf5
Epoch 3/50
 - 1s - loss: 0.5323 - acc: 0.7454 - val_loss: 0.5316 - val_acc: 0.7461

Epoch 00003: val_loss improved from 0.53617 to 0.53157, saving model to MLP_val_weights.hdf5
Epoch 4/50
 - 1s - loss: 0.5251 - acc: 0.7506 - val_loss: 0.5284 - val_acc: 0.7470

Epoch 00004: val_loss improved from 0.53157 to 0.52842, saving model to MLP_val_weights.hdf5
Epoch 5/50
 - 1s - loss: 0.5204 - acc: 0.7534 - val_loss: 0.5267 - val_acc: 0.7481

Epoch 00005: val_loss improved from 0.52842 to 0.52672, saving model to MLP_val_weights.hdf5
Epoch 6/50
 - 1s - loss: 0.5157 - acc: 0.7550 - val_loss: 0.5280 

#################################
Number of units: 64
Batch size: 256
Dropout rate 1: 0.21523440914624525
Dropout rate 2: 0.17584236813506005
Fold: 0
Best val loss: 0.5267187065548367
#################################
Time to train MLP:  61.42967367172241
Entire process took:  61.51646947860718
(10096, 2)


In [17]:
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(1)


num_per_fold = x_train.shape[0]//10
print(num_per_fold)

start_time = time.time()

n_units = 64
b_size = 256
dropout_rate_1 = 0.21523440914624525
dropout_rate_2 = 0.17584236813506005

exp_record = []

for f in range(1):#10
    #start = f*num_per_fold
    end = (f+1)*num_per_fold
    train1_x = x_train[:end, ]
    train2_x = x_train[end:, ]
    train_x_mlp = np.append(train2_x, train1_x, axis=0) #new obj
    train1_y = y_train[:end, ]
    train2_y = y_train[end:, ]
    train_y_mlp = np.append(train2_y, train1_y, axis=0)

    start_time_inner = time.time()
    with tf.Session(config = tf.ConfigProto(log_device_placement = True)):
        model = set_up_model(n_units, 
                             dropout_rate_1, 
                             dropout_rate_2)
        v_loss, val_weight = fit_model(model, 
                                       train_x_mlp, 
                                       train_y_mlp, 
                                       b_size)
        print("#################################")
        print("Number of units:", n_units)
        print("Batch size:", b_size)
        print("Dropout rate 1:", dropout_rate_1)
        print("Dropout rate 2:", dropout_rate_2)
        print("Fold:", f)
        print("Best val loss:", v_loss)
        exp_record.append([n_units, b_size, 
                           dropout_rate_1, 
                           dropout_rate_2, 
                            f, v_loss])
        print("#################################")
        if(f==4):
            print(exp_record)
        model.load_weights(val_weight)
        yhat_test = model.predict(x_test)
        print("Time to train MLP: ", time.time() - start_time_inner)
print("Entire process took: ", time.time() - start_time)
print(yhat_test.shape)

In [18]:
print(yhat_test.shape)
yhat_test_df = pd.DataFrame(yhat_test)
y_test_df = pd.DataFrame(y_test)
yhat_test_df.columns = ['NON_ADHERENT', 'ADHERENT']
y_test_df.columns = ['NON_ADHERENT', 'ADHERENT']
feather.write_dataframe(yhat_test_df, "/home/whsu014/data/PDC_MLP_yhat.feather")
feather.write_dataframe(y_test_df, "/home/whsu014/data/PDC_y_MLP.feather")

(10096, 2)
